In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
# with help from http://planspace.org/20150423-forward_selection_with_statsmodels/
import statsmodels.formula.api as smf

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response, ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < (best_new_score * 1):
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response, ' + '.join(selected))
    
    model = smf.ols(formula, data).fit()
    return model

In [22]:
response=pd.read_pickle('data/ZipcodeMVPSnew.pkl')[['RegionName','2000_agg','2010_agg']]
response=response.rename(columns={'RegionName': 'zipcode'})
response['2010_agg_adj'] = response['2010_agg']/1.26
response['pct_delta'] = (response['2010_agg_adj']/response['2000_agg']) - 1
response = response.drop(['2010_agg','2000_agg', '2010_agg_adj'], 1)

In [23]:
response.head(3)

,zipcode,pct_delta
1,79936,0.173843
2,60629,0.085907
3,90650,0.426587


In [24]:
# training and test sets that were normalized, and training and test sets that were normalized and transformed ('_t')
train_set = pd.read_pickle('data/train_all_features_norm.pkl')
itrain_df = pd.DataFrame(train_set.index).rename(columns={0: 'ind'})

train_set_t = pd.read_pickle('data/train_all_features_norm_and_transform.pkl')
itrain_t_df = pd.DataFrame(train_set_t.index).rename(columns={0: 'ind'})

test_set = pd.read_pickle('data/test_all_features_norm.pkl')
itest_df = pd.DataFrame(test_set.index).rename(columns={0: 'ind'})

test_set_t = pd.read_pickle('data/test_all_features_norm_and_transform.pkl')
itest_t_df = pd.DataFrame(test_set_t.index).rename(columns={0: 'ind'})

features = pd.concat([train_set, test_set])
features_t = pd.concat([train_set_t, test_set_t])

print features.shape, train_set.shape, test_set.shape
features.head(10)

(27653, 64) (22122, 64) (5531, 64)


,zipcode,prison,jail,A001,A002,A003,A004,A005,A006,A007,A008,A009,A010,A011,A012,A013,A014,A015,A016,A017,A018,A019,A020,A021,A022,A023,A024,A025,A026,A027,A028,A029,A030,A031,A032,A033,A034,A035,A036,A037,A038,A039,A040,A041,A042,A043,A044,A045,A046,A047,A048,A049,A050,A051,A052,A053,A054,A055,A056,n_establishments,paid_employees,first_quarter_payroll_1000,annual_payroll_1000,median_income
0,01001,0,0,0.393159,-0.844788,-0.576017,-0.174118,-0.117701,0.107539,-0.016376,0.170177,0.302420,1.862902,-0.661973,-0.790923,-0.635659,-0.198994,-0.050199,-0.043662,-0.090009,-0.044475,-0.020593,1.102067,0.725830,-0.772259,-0.455802,-0.057597,-0.171337,0.315046,0.117725,0.383984,0.672488,2.097870,0.537700,-0.396109,-0.176487,-0.012918,-0.132979,-0.367899,-0.890846,-0.317130,-0.457076,-0.174062,-0.299229,1.122282,0.268374,0.452518,1.600822,1.382658,-0.747222,0.588576,0.758788,-0.797671,-0.140852,-0.530444,-0.652569,-0.228583,0.486598,-1.127662,0.459736,0.742016,0.244980,0.334447,0.334018
1,01002,0,0,1.245532,-1.599994,1.060064,3.597249,-1.353027,-2.046088,-1.219645,-0.868623,-0.806043,-0.218702,-0.400512,-1.502592,0.492105,2.820575,-1.110719,-1.885773,-1.384025,-0.915197,-0.790664,-0.376806,0.464421,-1.508369,1.426019,3.888342,-1.261798,-1.604638,-0.859521,-0.715226,-0.731682,-0.143021,-0.223040,-0.195494,-0.162262,1.524147,-0.132979,-0.065958,-1.612656,-1.106050,-0.983398,-0.122170,0.166579,0.116185,-0.327456,-0.507041,0.520319,0.090974,-0.843631,-0.630502,0.751994,-0.790133,-0.264675,-0.530444,-0.166848,-0.330651,0.973388,-0.031431,0.763299,1.633192,0.335312,0.417431,0.110293
3,01005,0,0,-0.418261,-0.321953,0.815872,-0.337381,-0.276076,0.681839,0.556609,-0.311218,-0.667485,0.078670,0.165987,-0.298230,1.099363,-0.272642,-0.387638,0.416865,0.479358,0.001353,-0.469801,-0.202821,-0.101965,-0.262645,0.416261,-0.352621,-0.107192,0.778418,0.552056,-0.565334,-0.865412,0.193112,0.667700,-0.433724,-0.176487,-0.316536,-0.132979,-0.466359,0.196325,0.156222,0.244687,-0.139468,-0.066325,-0.223221,-0.142543,0.185974,-0.233994,0.090974,0.291852,-0.171839,0.602543,-0.624302,-0.471047,0.240861,0.226354,0.230722,-0.461362,-0.442518,-0.416119,-0.399458,-0.271116,-0.314831,0.984848
4,01007,0,0,0.246007,0.171836,0.254233,-0.435338,0.008999,1.327927,0.757154,-0.361891,-0.736764,-0.553246,-0.247993,0.194464,0.231852,-0.346290,-0.194816,0.704695,0.582879,-0.365267,-0.790664,-0.463799,0.311932,0.133722,0.232669,-0.463255,0.341821,1.572770,0.823513,-0.365477,-0.664817,-0.535177,0.547330,-0.421186,-0.190712,-0.012918,-0.003289,-0.413847,0.321082,-0.685293,-0.457076,-0.104873,0.195692,-0.502019,-0.656190,-0.507041,-0.172834,-0.339587,0.538230,-0.835693,0.779167,-0.820284,-0.677419,-0.221922,0.249484,0.154171,0.191188,-0.481669,-0.062791,-0.286955,-0.233152,-0.263373,1.043897
6,01009,0,0,-0.719368,-0.496231,-0.087635,0.201386,0.832550,0.574158,-0.102324,-0.108525,-1.256356,0.487556,0.492813,-0.133999,0.492105,0.169246,0.962115,-0.043662,-0.245291,0.230490,-0.983182,0.145149,-0.428727,-0.828883,-0.685293,0.200548,0.341821,1.043202,0.009142,-0.365477,-1.333469,0.585268,0.658071,-0.477609,-0.190712,-0.335512,-0.132979,-0.453231,-0.561130,-0.001562,-0.457076,0.759997,0.894404,0.528321,0.658746,-0.027262,0.234903,0.029466,-0.286602,-0.618432,0.548198,-0.564000,-0.388498,-0.838966,-0.375014,-0.177549,-0.754192,-0.227187,-0.617665,-0.509473,-0.307238,-0.358285,-0.233389
9,01012,0,0,-0.724921,-1.193344,-0.160892,-0.435338,-0.592827,0.466476,1.817176,0.727583,-0.563567,0.190184,0.144198,-1.283617,0.058350,-0.125346,-0.725076,-0.331492,2.083938,0.642937,-0.469801,0.667104,-0.080181,-0.998755,-0.409904,-0.795156,-0.171337,1.307986,1.257844,0.733733,-0.531086,-0.086999,0.720663,-0.490147,-0.176487,-0.259608,-0.132979,-0.486050,-0.222503,0.839952,2.525414,-0.398929,-0.677697,0.128307,0.206737,-0.773585,-0.009739,-0.831657,-0.393723,-0.654642,-0.124331,0.182238,-0.759968,-0.838966,-0.398144,-0.407202,-0.753849,0.203475,-0.622642,-0.515830,-0.312899,-0.364498,-0.092024
11,01020,0,0

In [25]:
merged = response.merge(features, how='inner', on=['zipcode'])
merged_t = response.merge(features_t, how='inner', on=['zipcode'])

merged = merged.astype('float')
merged_t = merged_t.astype('float')

merged = merged.rename(columns={'pct_delta' : 'response'})
merged_t = merged_t.rename(columns={'pct_delta' : 'response'})

merged_t.shape, merged.shape

((11602, 65), (11602, 65))

In [26]:
merged_train = merged.merge(itrain_df, how='inner', right_on='ind', left_index=True)
merged_test = merged.merge(itest_df, how='inner', right_on='ind', left_index=True)
merged_t_train = merged_t.merge(itrain_t_df, how='inner', right_on='ind', left_index=True)
merged_t_test = merged_t.merge(itest_t_df, how='inner', right_on='ind', left_index=True)

In [27]:
merged_train.shape, merged_test.shape, merged_t_train.shape, merged_t_train.shape

((9271, 66), (2331, 66), (9271, 66), (9271, 66))

In [28]:
import time
start_time = time.time()
bestmodel = forward_selected(merged_train, 'response')

print '%s seconds to train model'%(time.time() - start_time)

93.6992521286 seconds to train model


In [29]:
bestmodel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               response   R-squared:                       0.390
Model:                            OLS   Adj. R-squared:                  0.387
Method:                 Least Squares   F-statistic:                     120.3
Date:                Mon, 07 Dec 2015   Prob (F-statistic):               0.00
Time:                        14:21:30   Log-Likelihood:                 2154.7
No. Observations:                9271   AIC:                            -4209.
Df Residuals:                    9221   BIC:                            -3853.
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------------
Intercept                     -0.5131      0.567     -0.905      0.366        -1.625     0.599
A002                          -0.0550      0.008     -6.773      0.000        -0.071    -0.039
A054                          -0.0208      0.011     -1.906      0.057        -0.042     0.001
A003                          -0.0848      0.016     -5.374      0.000        -0.116    -0.054
median_income                  0.0532      0.004     13.315      0.000         0.045     0.061
A050                          16.3152     12.800      1.275      0.202        -8.776    41.407
A052                          -0.0617      0.004    -16.821      0.000        -0.069    -0.055
A024                          -0.1257      0.077     -1.637      0.102        -0.276     0.025
A051                          -0.0435      0.003    -12.660      0.000        -0.050    -0.037
zipcode                    -1.765e-06   9.04e-08    -19.528      0.000     -1.94e-06 -1.59e-06
A036                           0.0224      0.005      4.621      0.000         0.013     0.032
A035                           0.0120      0.002      5.559      0.000         0.008     0.016
A031                          -0.1226      0.015     -8.274      0.000        -0.152    -0.094
ind                         6.755e-06   1.73e-06      3.896      0.000      3.36e-06  1.02e-05
A032                          -0.0991      0.012     -8.111      0.000        -0.123    -0.075
A044                           0.0389      0.010      4.085      0.000         0.020     0.058
A029                           0.0985      0.044      2.237      0.025         0.012     0.185
A040                           0.0805      0.016      5.194      0.000         0.050     0.111
annual_payroll_1000           -0.0106      0.016     -0.674      0.500        -0.041     0.020
n_establishments               0.0554      0.005     10.665      0.000         0.045     0.066
paid_employees                -0.0505      0.007     -7.022      0.000        -0.065    -0.036
A041                          -0.0564      0.013     -4.470      0.000        -0.081    -0.032
A019                           0.1859      0.046      4.061      0.000         0.096     0.276
A037                          -0.2038      0.016    -12.690      0.000        -0.235    -0.172
A042                          -0.5743      0.327     -1.756      0.079        -1.215     0.067
A048                           0.1330      0.016      8.513      0.000         0.102     0.164
A047                           0.0934      0.015      6.030      0.000         0.063     0.124
A046                          -0.0629      0.011     -5.791      0.000        -0.084    -0.042
A034                          -0.0184      0.005     -4.044      0.000        -0.027    -0.009
A009                          -0.3239      

In [30]:
import time
start_time = time.time()
bestmodel_t = forward_selected(merged_t_train, 'response')

print '%s seconds to train model'%(time.time() - start_time)

79.7794640064 seconds to train model


In [31]:
bestmodel_t.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               response   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.398
Method:                 Least Squares   F-statistic:                     131.5
Date:                Mon, 07 Dec 2015   Prob (F-statistic):               0.00
Time:                        14:22:50   Log-Likelihood:                 2240.8
No. Observations:                9271   AIC:                            -4386.
Df Residuals:                    9223   BIC:                            -4043.
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------
Intercept               0.2113      0.020     10.737      0.000         0.173     0.250
A002                   -0.0510      0.009     -5.598      0.000        -0.069    -0.033
A036                    0.0668      0.004     18.148      0.000         0.060     0.074
A052                   -0.0622      0.004    -16.882      0.000        -0.069    -0.055
zipcode             -2.016e-06   9.37e-08    -21.522      0.000      -2.2e-06 -1.83e-06
A035                    0.0186      0.002     10.317      0.000         0.015     0.022
median_income           0.0400      0.004      9.845      0.000         0.032     0.048
A049                   -0.0702      0.005    -15.559      0.000        -0.079    -0.061
A005                    0.0381      0.010      3.808      0.000         0.018     0.058
A051                   -0.0390      0.004    -10.582      0.000        -0.046    -0.032
A037                   -0.1815      0.016    -11.447      0.000        -0.213    -0.150
A053                    0.0612      0.027      2.257      0.024         0.008     0.114
A018                    0.0438      0.013      3.332      0.001         0.018     0.070
paid_employees         -0.1167      0.011    -10.842      0.000        -0.138    -0.096
A040                    0.1019      0.015      6.915      0.000         0.073     0.131
A032                   -0.0365      0.005     -7.794      0.000        -0.046    -0.027
A034                    0.0200      0.004      5.689      0.000         0.013     0.027
n_establishments        0.1502      0.013     11.818      0.000         0.125     0.175
ind                  8.086e-06   2.02e-06      4.001      0.000      4.12e-06   1.2e-05
A033                    0.0558      0.010      5.357      0.000         0.035     0.076
A003                   -0.0693      0.015     -4.665      0.000        -0.098    -0.040
A042                   -0.6239      0.324     -1.928      0.054        -1.258     0.010
A044                    0.0398      0.009      4.204      0.000         0.021     0.058
A024                    0.0245      0.011      2.253      0.024         0.003     0.046
A041                   -0.0730      0.012     -6.011      0.000        -0.097    -0.049
A047                    0.1013      0.017      6.007      0.000         0.068     0.134
A055                   -0.0525      0.013     -3.951      0.000        -0.079    -0.026
A056                   -0.0244      0.006     -3.966      0.000        -0.037    -0.012
A031                   -0.0240      0.006     -3.897      0.000        -0.036    -0.012
annual_payroll_1000    -0.0060      0.002     -2.517      0.012        -0.011    -0.001
A030                    0.0199      0.008      2.489      0.013         0.004     0.036
A010                   -0.0343      0.010     -3.359      0.001        -0.054    -0.014
A04

In [32]:
best_features = bestmodel.params
best_features_t = bestmodel_t.params

In [33]:
len(merged_train.columns), len(best_features), len(best_features_t)

(66, 50, 48)

In [34]:
bad_features_t = [el for el in merged_t_train.columns if (el not in best_features_t.index.values.tolist())]
merged_train_t_best = merged_t_train.drop(bad_features_t + ['ind'], 1)
merged_test_t_best = merged_t_test.drop(bad_features_t + ['ind'], 1)

In [36]:
merged_train_t_best.head(2)

,zipcode,jail,A002,A003,A005,A009,A010,A013,A015,A016,A018,A019,A024,A027,A028,A029,A030,A031,A032,A033,A034,A035,A036,A037,A038,A039,A040,A041,A042,A043,A044,A045,A046,A047,A048,A049,A051,A052,A053,A054,A055,A056,n_establishments,paid_employees,annual_payroll_1000,median_income
0,79936,0,1.043228,1.353093,0.610825,-1.290995,-0.924961,1.22949,0.142623,-0.21636,-1.235989,-1.239872,0.495572,-0.370898,-1.164903,-1.199738,-1.299125,0.008071,-0.023733,-0.119587,-0.020352,-0.003289,2.706301,1.256761,0.471790,0.420127,1.590272,1.826019,-1.362656,-1.087653,-1.200055,-1.212563,-1.354481,2.230742,-1.402987,0.860686,-0.729692,-0.376183,1.637257,1.481053,1.805892,1.064799,1.825619,1.803127,1.422776,-0.054825
1,60629,0,1.711295,1.035645,1.180975,-1.325635,-0.813446,0.83911,0.913910,-0.50419,-1.327644,-1.239872,0.864351,-1.185269,-1.314796,-1.266604,-0.929836,-1.994890,1.530919,-0.105362,-0.396648,-0.132979,2.470766,0.909223,2.049629,1.472771,2.040004,1.796906,-0.938398,-0.758919,-0.507041,-0.825213,-0.862411,2.359287,-0.871903,0.290055,1.099626,0.240861,2.909382,3.165172,1.779076,1.573763,1.350804,1.178124,0.430216,-0.457911


## Linear Regression with Lasso Regularization (normalized and transformed features)

In [50]:
from sklearn.linear_model import LassoCV
# from sklearn.linear_model import Lasso
# from sklearn.grid_search import GridSearchCV

# clf_lasso = Lasso()
# alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

start_time = time.time()

X_train = merged_train_t_best.drop('zipcode', 1).values
Y_train = merged_train_t_best['zipcode'].values
X_test = merged_test_t_best.drop('zipcode', 1).values
Y_test = merged_test_t_best['zipcode'].values

# start_time = time.time()

# n_folds = 5
# n_jobs = 4
# gs = GridSearchCV(clf_lasso, param_grid={"alpha": alphas}, cv=n_folds, n_jobs=n_jobs)
# gs.fit(X_train, Y_train)

# alpha_best = gs.best_params_['alpha']
# print 'The best alpha-value is %s'%(alpha_best)

# print '----- %s seconds to run -----'%(time.time() - start_time)

alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
alphas_temp = [0.001, 0.01]
clf_lasso_model = LassoCV(alphas = alphas).fit(X_train, Y_train)
best_alpha_lasso = clf_lasso_model.alpha_

print '----- %s seconds to run ----- \n---- best alpha = %s ----'%(time.time() - start_time, best_alpha_lasso)

----- 0.226511955261 seconds to run ----- 
---- best alpha = 100.0 ----


In [51]:
# calculate accuracy
start_time = time.time()

training_accuracy = clf_lasso_model.score(X_train, Y_train)
test_accuracy = clf_lasso_model.score(X_test, Y_test)
print '#### based on the best Linear Regression with Lasso Regularization ####'
print "R-squared on training data: %0.2f" % (training_accuracy)
print "R-squared on test data:     %0.2f" % (test_accuracy)
print '----- %s seconds to run -----'%(time.time() - start_time)

#### based on the best Linear Regression with Lasso Regularization ####
R-squared on training data: 0.51
R-squared on test data:     0.51
----- 0.00239086151123 seconds to run -----


## Linear Regression with Ridge Regularization (normalized and transformed features)

In [52]:
from sklearn.linear_model import RidgeCV

start_time = time.time()

alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
clf_ridge_model = RidgeCV(alphas = alphas).fit(X_train, Y_train)
best_alpha_ridge = clf_ridge_model.alpha_

print '----- %s seconds to run ----- \n---- best alpha = %s ----'%(time.time() - start_time, best_alpha_ridge)

----- 0.0566458702087 seconds to run ----- 
---- best alpha = 10.0 ----


In [53]:
training_accuracy = clf_ridge_model.score(X_train, Y_train)
test_accuracy = clf_ridge_model.score(X_test, Y_test)
print '#### based on the best Linear Regression with Ridge Regularization ####'
print "R-squared on training data: %0.2f" % (training_accuracy)
print "R-squared on test data:     %0.2f" % (test_accuracy)

#### based on the best Linear Regression with Ridge Regularization ####
R-squared on training data: 0.52
R-squared on test data:     0.52


In [ ]:
##### In case we want to do SVM regression/other types of models later

In [ ]:
from sklearn.svm import LinearSVC

clfsvm=LinearSVC(loss="hinge")
Cs=[0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
Xmatrix=dftouse[lcols].values
Yresp=dftouse['RESP'].values

Xmatrix_train=Xmatrix[mask]
Xmatrix_test=Xmatrix[~mask]
Yresp_train=Yresp[mask]
Yresp_test=Yresp[~mask]

#your code here
from sklearn.grid_search import GridSearchCV
import time

start_time = time.time()

n_folds = 5
n_jobs = 4
gs = GridSearchCV(clfsvm, param_grid={"C": Cs}, cv=n_folds, n_jobs=n_jobs)
gs.fit(Xmatrix_train, Yresp_train)

C_best = gs.best_params_['C']
print 'The best c-value is %s'%(C_best)

print '----- %s seconds to run -----'%(time.time() - start_time)

In [ ]:
#calculate the accuracy here
#your code here
start_time = time.time()

best = LinearSVC(C=C_best, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
      penalty='l2', random_state=None, tol=0.0001, verbose=0)
best.fit(Xmatrix_train, Yresp_train)

training_accuracy = best.score(Xmatrix_train, Yresp_train)
test_accuracy = best.score(Xmatrix_test, Yresp_test)
print '#### based on the best LinearSVC ####'
print "Accuracy on training data: %0.2f" % (training_accuracy)
print "Accuracy on test data:     %0.2f" % (test_accuracy)
print '----- %s seconds to run -----'%(time.time() - start_time)